# 05 Penggugusan

Oleh: Murthadza bin Aznam <br>
Tarikh: 9th August 2021<br>

Nota interaktif ini ditulis sebagai sebahagian daripada kursus _International Virtual Conference on Astrostatistics and Machine Learning_ (IVCASML2021).

---

## 0.0 MATLAMAT
1. Ingin menentukan bintang-bintang segugus menggunakan algoritma HDBSCAN

---
---
# ! PERINGATAN !
Nota interaktif ini adalah nota yang berat. Nota ini akan menjadi makin lembab semakin jauh kita bergerak ke bawah. Aku syak kerana banyak data yang perlu dipegang jadi aku sudah letak beberapa arahan `del` sepanjang nota ini untuk buang data yang tidak lagi diperlukan. Walaupun begitu, ia masih berat. 

---
---

## 1 PERSIAPAN

### 1.1 PERSIAPAN: Mengimport pakej

In [ ]:
import math
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import hdbscan
from sklearn.preprocessing import StandardScaler

### 1.2 PREPARATION: Tetapan yang digunakan untuk plot

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # Saiz lalai teks
plt.rc('axes', titlesize=SMALL_SIZE)     # Saiz teks untuk paksi
plt.rc('axes', labelsize=MEDIUM_SIZE)    # Saiz teks untuk label x dan y
plt.rc('xtick', labelsize=SMALL_SIZE)    # Saiz label untuk senggat x
plt.rc('ytick', labelsize=SMALL_SIZE)    # Saiz label untuk senggat y
plt.rc('legend', fontsize=10)            # Saiz teks untuk panduan graf

%matplotlib inline

### 1.3 PERSIAPAN: Persiapan Data
Terdapat data dari tiga gugusan bintang:
1. NGC725
2. M67
3. NGC7789

Nota ini hanya boleh proses satu per satu, jadi keluarkan dari komen mana-mana bintang yang berkenan dan masukkan ke komen untuk bintang yang tidak mahu diproses. Data ini diperoleh dari Gaia EDR3.

In [ ]:
DATADIR = "../dataset/"

# FILENAME = "gaiaedr3_180_NGC752"
# CLUSTER = "NGC752"

FILENAME = "gaiaedr3_150_M67"
CLUSTER = "M67"

# FILENAME = "gaiaedr3_45_NGC7789"
# CLUSTER = "NGC7789"

OUTPUTDIR = "../output/"

datafile = pd.read_csv(DATADIR + FILENAME + ".csv", delimiter=",")
datafile.head()

### 1.4 PERSIAPAN: Mengabaikan data yang tiada

In [ ]:
datafile = datafile.dropna(subset=['pmra', 'pmdec', 'parallax']).reset_index()

Untuk membuang data-data yang mempunyai ketidakpastian yang tinggi tetapi dalam masa yang sama inginkan data dengan magnitud G lebih kurang 21 mag, kita pilih ralat yang kurang daripada 0.005 dalam data G-mag masing-masing. Kira ralat G ($|\sigma_G|$), $G_{BP}$ ($|\sigma_{BP}|$), dan $G_{RP}$ ($|\sigma_{RP}|$).

$$\begin{array}
|\sigma_G| &= -\frac{2.5}{ln \ 10} \frac{\sigma_{F_G}}{F_G} \\
|\sigma_{BP}| &= -\frac{2.5}{ln \ 10} \frac{\sigma_{F_{BP}}}{F_{BP}} \\
|\sigma_{RP}| &= -\frac{2.5}{ln \ 10} \frac{\sigma_{F_{RP}}}{F_{RP}} \\
\end{array}$$

In [ ]:
datafile['e_Gmag'] = abs(-2.5*datafile['phot_g_mean_flux_error']/math.log(10)/datafile['phot_g_mean_flux'])
datafile['e_BPmag'] = abs(-2.5*datafile['phot_bp_mean_flux_error']/math.log(10)/datafile['phot_bp_mean_flux'])
datafile['e_RPmag'] = abs(-2.5*datafile['phot_rp_mean_flux_error']/math.log(10)/datafile['phot_rp_mean_flux'])

In [ ]:
# Ini adalah warna bintang. Dalam astronomi, warna bintang ditakrifkan sebagai perbezaan magnitud dalam jalur cahaya yang berbeza
datafile['bp_rp'] = datafile['phot_bp_mean_mag'] - datafile['phot_rp_mean_mag']

Pilih data dengan nilai paralaks yang positif ($\omega>0$) dan ralat magnitud G ($\sigma_G$) $< 0.005$.

In [ ]:
data = datafile[(datafile['parallax'] > 0.) & (datafile['e_Gmag'] < 0.005)].reset_index(drop=True)

del datafile

## 2 PLOT AWAL

### 2.1 PLOT AWAL: Taburan Dalam Ruang

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.plot(data['ra'], data['dec'], ',')

plt.xlabel(r'$\alpha$ (darjah)')
plt.ylabel(r'$\delta$ (darjah)')

plt.show()

### 2.2 PLOT AWAL: Rajah Titik Vektor

In [ ]:
# Ini adalah halaju bintang
# Pergerakan wajar dalam arah jarak hamal melawan pergerakan wajar dalam arah keserongan
fig = plt.figure(figsize=(6, 6))
plt.plot(data['pmra'], data['pmdec'], ',')

# mas = miliarksaat
plt.xlabel(r'$\mu_{\alpha*}$ (mas/tahun)')
plt.ylabel(r'$\mu_{\delta}$ (mas/tahun)')

# Bintang dalam satu gugus bergerak dalam halaju yang sama jadi kita tumpu pada satu kawasan kecil
plt.xlim(-30,30)
plt.ylim(-30,30)

plt.show()

### 2.3 PLOT AWAL: Rajah Magnitud melawan Warna

In [ ]:
fig = plt.figure(figsize=(6, 8))
plt.plot(data['bp_rp'], data['phot_g_mean_mag'], ',')

ax = plt.gca()
ax.invert_yaxis()
plt.xlim(0., 3.)

plt.xlabel('bp - rp') # Warna
plt.ylabel('g') # Magnitud

plt.show()

## 3 HDBSCAN
Penggugusan Ruang Bersandarkan Ketumpatan Berhierarki (HDBSCAN: _Hierarchichal Density--Based Spatial Clustering of Applications with Noise_) menggunakan ketumpatan titik data untuk menentukan sama ada titik tersebut layak atau tidak masuk dalam gugusan.

### 3.1 HDBSCAN: Persiapan data

Kita akan gunakan titik-titik daripada **pergerakan wajar dalam arah jarak hamal**, **pergerakan wajar dalam arah keserongan** dan **paralaks** untuk menentukan gugusan bintang kerana setiap bintang dalam gugusan yang sama sepatutnya mempunyai nilai-nilai ini yang sama atau hampir.

In [ ]:
# Kita pilih pembolehubah yang dikehendaki
df = data[["pmra", "pmdec", "parallax"]]
df = df.to_numpy().astype("float32", copy = False)

stscaler_df = StandardScaler().fit(df)
df_ = stscaler_df.transform(df)

### 3.2 HDBSCAN: Melaksanakan algoritma HDBSCAN

In [ ]:
clus_size = 2 * df_.shape[1]

clusterer = hdbscan.HDBSCAN(clus_size)
cluster_labels = clusterer.fit_predict(df_)

data['hdbscan'] = cluster_labels

del df, df_, cluster_labels

### 3.3 HDBSCAN: Menyemak Label

Warna yang diletakkan pada titik-titik tersebut menunjukkan label yang dikenakan padanya.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
gr = ax.scatter(data['pmra'], data['pmdec'], s=10, c=data['hdbscan'])#, edgecolor='')

fig.colorbar(gr, ax=ax)
ax = plt.gca()
ax.invert_yaxis()
plt.xlim(-30,30)
plt.ylim(-30,30)

plt.xlabel(r'$\mu_{\alpha*}$ (mas/tahun)')
plt.ylabel(r'$\mu_{\delta}$ (mas/tahun)')

plt.show()

Taburan setiap label.

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(data['hdbscan'])

plt.xlabel('Label Kluster')
plt.ylabel('Jumlah Sumber')

plt.show()

Jumlah ahli dalam kluster

In [ ]:
data['hdbscan'].value_counts()

### 3.4 HDBSCAN: Pembersihan

Buang data yang dilabelkan sebagai bukan ahli (`label = -1`).

In [ ]:
result_hdbscan = data[data['hdbscan'] >= 0].reset_index(drop=True)

c = result_hdbscan['hdbscan'].value_counts()
del result_hdbscan

In [ ]:
n_max = c.index[np.argmax(c)]
del c

result = data[data['hdbscan'] == n_max]
result.to_csv(OUTPUTDIR + "results_" + CLUSTER + ".csv", index=False)

## 4 PLOT HASIL

### 4.1 HASIL: Taburan Ruang

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = plt.subplot()
plt.plot(data['ra'], data['dec'], '.', mec='silver', mfc='darkgray', markersize=1., label="Semua data")
plt.plot(result['ra'], result['dec'], 'o', mfc='tab:orange', markersize=2., label="HDBSCAN")

plt.xlabel(r'$\alpha$ (deg)')
plt.ylabel(r'$\delta$ (deg)')
plt.legend()
plt.title("Taburan ruang untuk kluster bintang " + CLUSTER)
plt.savefig(OUTPUTDIR + "ms " + FILENAME + "_spatial.jpg")
plt.show()

### 4.2 HASIL: Rajah Titik Vektor

Bintang-bintang segugus sepatutnya bergerak dalam halaju yang sama

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.plot(data['pmra'], data['pmdec'], '.', mec='silver', mfc='darkgray', markersize=5., label="Semua data")
plt.plot(result['pmra'], result['pmdec'], 'o', mfc='tab:orange', mec='None', markersize=5., label="HDBSCAN")

plt.xlabel(r'$\mu_{\alpha*}$ (mas/tahun)')
plt.ylabel(r'$\mu_{\delta}$ (mas/tahun)')

plt.xticks()
plt.yticks()

XMED = np.median(result['pmra'])
YMED = np.median(result['pmdec'])

XVAR = np.median(abs(result['pmra']-XMED))
YVAR = np.median(abs(result['pmdec']-YMED))

VAR = (XVAR + YVAR)/2

plt.xlim(XMED+6*VAR,XMED-6*VAR)
plt.ylim(YMED+6*VAR,YMED-6*VAR)

del XMED, XVAR, YMED, YVAR, VAR

plt.legend()
plt.title("Rajah Titik Vektor untuk Gugusan Bintang " + CLUSTER)

plt.savefig(OUTPUTDIR + "ms " + FILENAME + "_vector_point.jpg")
plt.show()

### 4.3 RESULTS: Rajah Magnitud melawan Warna

Bintang-bintang segugus sepatutnya membentuk Rajah HR

In [ ]:
plt.figure(figsize=(6, 8))
plt.plot(data['bp_rp'], data['phot_g_mean_mag'], '.', mec='silver', mfc='darkgray', markersize=2., label="Semua data")
plt.plot(result['bp_rp'], result['phot_g_mean_mag'], 'o', color='tab:orange', markersize=2., label="HDBSCAN")

plt.xlabel(r'$G_{BP}-G_{RP}$')
plt.ylabel(r'$G$ (mag)')

plt.xlim(0., 3.)
plt.gca().invert_yaxis()
plt.legend()
plt.title("Rajah Magnitud melawan Warna untuk Gugusan Bintang " + CLUSTER)

plt.savefig(OUTPUTDIR + "ms " + FILENAME + "_color_magnitude.jpg")
plt.show()

### 4.4 HASIL: Taburan Paralaks

Bintang-bintang segugus sepatutunya berada dalam jarak yang sama (iaitu ada paralaks yang sama atau hampir)

In [ ]:
bins_all = np.arange(data['parallax'].min(), data['parallax'].max(), .01)
bins_sam = np.arange(result['parallax'].min(), result['parallax'].max(), .01)

In [ ]:
plt.figure(figsize=(6, 4))
data.parallax.hist(bins=bins_all, color='gray', label="Semua data")
result.parallax.hist(bins=bins_sam, color='orange', label="HDBSCAN")

del bins_all, bins_sam

plt.xlabel(r'$\omega$ (mas)')
plt.ylabel('Jumlah sumber')

plt.xlim(0, 5)

plt.xticks()
plt.yticks()

plt.legend()
plt.title("Taburan Data untuk Gugusan Bintang " + CLUSTER)

plt.savefig(OUTPUTDIR + "ms " +FILENAME + "_parallax_dist.jpg")
plt.show()

## 5 TUGASAN TAMBAHAN

### 5.1 Tentukan titik tengah pada gugusan bintang

In [ ]:
ra_c    = np.median(result['ra'])
dec_c   = np.median(result['dec'])
pmra_c  = np.median(result['pmra'])
pmdec_c = np.median(result['pmdec'])

fig = plt.figure(figsize=(6, 6))
ax = plt.subplot()
plt.plot(data['ra'], data['dec'], '.', mec='silver', mfc='darkgray', markersize=1., label="Semua data")
plt.plot(result['ra'], result['dec'], 'o', mfc='tab:orange', markersize=2., label="HDBSCAN")
plt.plot(ra_c, dec_c, 'x')

plt.xlabel(r'$\alpha$ (darjah)')
plt.ylabel(r'$\delta$ (darjah)')
plt.legend()
plt.show()

### 5.2 Halaju titik tengah gugusan

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.plot(data['pmra'], data['pmdec'], '.', mec='silver', mfc='darkgray', markersize=5., label="Semua data")
plt.plot(result['pmra'], result['pmdec'], 'o', mfc='tab:orange', mec='None', markersize=5., label="HDBSCAN")
plt.plot(pmra_c, pmdec_c, '+')

plt.xlabel(r'$\mu_{\alpha*}$ (mas/yr)')
plt.ylabel(r'$\mu_{\delta}$ (mas/yr)')

plt.xticks()
plt.yticks()

XMED = np.median(result['pmra'])
YMED = np.median(result['pmdec'])

XVAR = np.median(abs(result['pmra']-XMED))
YVAR = np.median(abs(result['pmdec']-YMED))

VAR = (XVAR + YVAR)/2

plt.xlim(XMED+6*VAR,XMED-6*VAR)
plt.ylim(YMED+6*VAR,YMED-6*VAR)

del XMED, XVAR, YMED, YVAR, VAR

plt.legend()
plt.show()

## 6. Rujukan
Berikut merupakan senarak rujukan yang disediakan oleh pensyarah

1. Agarwal, Manan, et al. "ML-MOC: Machine Learning (kNN and GMM) based Membership determination for Open Clusters." Monthly Notices of the Royal Astronomical Society 502.2 (2021): 2582-2599.
2. Campello, Ricardo JGB, et al. "Hierarchical density estimates for data clustering, visualization, and outlier detection." ACM Transactions on Knowledge Discovery from Data (TKDD) 10.1 (2015): 1-51.
3. Chen, W. P., C. W. Chen, and C. G. Shu. "Morphology of Galactic open clusters." The Astronomical Journal 128.5 (2004): 2306.
4. Ester, Martin, et al. "A density-based algorithm for discovering clusters in large spatial databases with noise." kdd. Vol. 96. No. 34. 1996.
5. Gaia Collaboration. "VizieR Online Data Catalog: Gaia DR2 (Gaia Collaboration, 2018)." VizieR Online Data Catalog (2018): I-345.
6. Kounkel, Marina, and Kevin Covey. "Untangling the Galaxy. I. Local Structure and Star Formation History of the Milky Way." The Astronomical Journal 158.3 (2019): 122.
7. McInnes, Leland, John Healy, and Steve Astels. "hdbscan: Hierarchical density based clustering." Journal of Open Source Software 2.11 (2017): 205.
8. Pedregosa, Fabian, et al. "Scikit-learn: Machine learning in Python." the Journal of machine Learning research 12 (2011): 2825-2830.










